In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Backtrader documentation
- simle example using backtrader
- https://ntguardian.wordpress.com/2017/06/12/getting-started-with-backtrader/ 
- 위 블로그를 참고해서 작성했다. 

In [2]:
#!pip3 install backtrader

     |████████████████████████████████| 419kB 5.0MB/s 


사용자가 backtesting system의 중요한 부분들, trading strategy, broker, sizers 등을 설정한다. 따라서 상당히 자율성이 보장되는 듯 하다. 

In [0]:
import backtrader as bt
import backtrader.indicators as btind
import datetime
import pandas as pd
from pandas import Series, DataFrame
import random
from copy import deepcopy
#from datetime import datetime

package import 후에 simple moving average crossover strategy(SMA) 전략을 사용해서 backtrader 알고리즘을 테스트 해 볼 예정이다. 


전략을 규정하기 위해서, 우선 중요한 indicators를 정의하고 초기화하는 __init()__ method를 설정해야 한다.   
그 뒤, 각 backtest bar에서 호출되는 next() method를 설정해야 한다.(buy, sell)    
더 복잡한 시스템은 log() method를 사용해서 next()에서 logging functions를 호출하게 만들어 logging result를 확인해야 한다. 


In [0]:
class SMAC(bt.Strategy):
  params = {'fast':20, 'slow':50, #the windows for both fast and slow moving average 
            'optim':False, 'optim_fs':(20, 50)} #used for optimization. first # = fast MA's window
                                                # second in tuple : slow MA's window
  def __init__(self):
    self.fastma = dict()
    self.slowma = dict()
    self.regime = dict()

    self._addobserver(True, bt.observers.BuySell) 

    if self.params.optim:
      self.params.fast, self.params.slow = self.params.optim_fs
    
    if self.params.fast > self.params.slow:
      raise ValueError(
          "A SMAC strategy cannot have the fast moving average's whindow"+\
          "greater than the slow moving average window"
      )

    for d in self.getdatanames():
      #the moving averages
      self.fastma[d] = btind.SimpleMovingAverage(self.getdatabyname(d),
                                                 period=self.params.fast,
                                                 plotname="FastMA: "+ d)
      self.slowma[d] = btind.SimpleMovingAverage(self.getdatabyname(d),
                                                 period=self.params.slow,
                                                 plotname="SlowMA"+d)
      self.regime[d] = self.fastma[d] - self.slowma[d]


  def next(self):
    for d in self.getdatanames(): #Looping through all symbols
      pos = self.getpositionbyname(d).size or 0
      if pos == 0: #Are we out or market?
        #consider the possibility of entrance
        # indexing! [0] : always means the present bar, [-1] the bar immediately preceding
        if self.regime[d][0] > 0 and self.regime[d][-1] <= 0:
          self.buy(data=self.getdatabyname(d))  #buy signal
      else:
        if self.regime[d][0] <= 0 and self.regime[d][-1] > 0:  # sell signal
          self.sell(data=self.getdatabyname(d))




Strategy를 규정한 이후에, SIzer라고 불리는 sizing object를 생성한다. 
Sizer 객체는 얼마나 구매/얼마나 판매할지를 결정하는 역할을 한다.   
여기에서는 그 시점에 포트폴리오의 10% 가치정도를 선택했다. 
또한 trades be done in batchs of 100 shares로 설정했다. 


In [0]:
class PropSizer(bt.Sizer):
  params = {'prop':0.1, 'batch':100}

  def _getsizing(self, comminfo, cash, data, isbuy):
    if isbuy: #buying
      target = self.broker.getvalue() * self.params.prop  #ideal total value of the position
      price = data.close[0]
      shares_ideal = target / price  #How many shares are needed to get target
      batches = int(shares_ideal / self.params.batch)
      shares = batches * self.params.batch

      if shares * price > cash : 
        return 0  #Not enough money for this trade

      else:
        return shares
    else:
      return self.broker.getposition(data).size  #clear the position

### Getting Cerebral with Cerebro

- Cerebro 객체는 backtest와 분석의 실행 역할을 하는 객체이다. 

In [0]:
cerebro = bt.Cerebro(stdstats=False)

In [0]:
cerebro.broker.set_cash(1000000)  #set our starting cash
cerebro.broker.setcommission(0.02)

get some money and give it to our 'broker' and also assign 2% commision to the broker

이제 Cerebro 객체에 넣을 데이터가 필요하다. 
pandas DataFrame, CSV files, databases, live data streams 모두 feed 가능하다. 

In [0]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2016, 10, 31)
is_first = True

symbols = ["AAPL", "GOOG", "AMZN", "NVDA"]
plot_symbols = ["AAPL", "GOOG", "NVDA"]

for s in symbols:
  data = bt.feeds.YahooFinanceData(dataname=s, fromdate = start, todate=end)
  if s in plot_symbols:
    if is_first:
      data_main_plot = data
      is_first = False
    else:
      data.plotinfo.plotmaster = data_main_plot
  else:
    data.plotinfo.plot = False
  cerebro.adddata(data)

이제 계좌 잔액을 추적하는 observer를 추가한다. 
observer : instance of what is known as an Observer object
Cerebro 객체가 생성되면 backtrader의 기본값은 자동적으로 Buy와 Sell order가 발생하면 계좌의 cash, value, occurrence of trades 3개의 값들을 추적하는 observers를 부착한다.


Observer 역시 next() method를 보유하고 있다. 
주의해야 하는 점   
index [0] : backtrader에서는 현재 step에서 현재 value를 의미한다. (today)    
index [-1] : 이전 value, 혹은 yesterday    
index[-2] : two days ago   
index [1] : tomorrow ... 


In [0]:
class AcctValue(bt.Observer):
  alias = ('Value',)
  lines = ('value',)

  plotinfo = {'plot':True, 'subplot':True}

  def next(self):
    self.lines.value[0] = self._owner.broker.getvalue()  #get today's account value (cash+stocks)

In [0]:
cerebro.addobserver(AcctValue)
cerebro.addstrategy(SMAC)
cerebro.addsizer(PropSizer)

In [36]:
cerebro.broker.getvalue()

1000000

In [37]:
cerebro.run()

In [38]:
cerebro.plot(iplot=True, volume=Falsec)

ImportError: ignored

In [39]:
cerebro.broker.getvalue()

1139128.4599999995